In [1]:
import time
import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split
import pandas as pd

In [2]:
SEED = 42
DIMENSIONS = (28, 28)
CHANNELS = 1  # grayscale
INPUT_SIZE = DIMENSIONS[0] * DIMENSIONS[1]
OUTPUT_SIZE = 10

def transform_data(
        is_convolutional: bool,
        x: pd.DataFrame,
        y: pd.Series | None = None) -> tuple[np.array, np.array]:
    """
    Transforms the data. Returns a tuple of (x, y) where x is a tensor of the features and y is a
    tensor of the labels.

    Args:
        is_convolutional: A boolean indicating whether the model is convolutional.
        x: A dataframe of the features.
        y: A series of the labels.
    """
    x = x.to_numpy()
    # Normalize the tensor
    x_min = x.min()
    x_max = x.max()
    x = (x - x_min) / (x_max - x_min)
    assert x.min() == 0
    assert x.max() == 1
    if is_convolutional:
        # Reshape data to have channel dimension
        # MNIST images are 28x28, so we reshape them to [batch_size, 1, 28, 28]
        x = x.reshape(-1, CHANNELS, DIMENSIONS[0], DIMENSIONS[1])
    if y is not None:
        y = y.to_numpy().astype(int)
    return x, y


def get_data():  # noqa
    """Function is required by and called from `model_pipeline()`."""
    x, y = fetch_openml('mnist_784', version=1, return_X_y=True, parser='auto')
    x, y = transform_data(is_convolutional=True, x=x, y=y)
    # 80% train; 10% validation; 10% test
    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=SEED)
    x_test, x_val, y_test, y_val = train_test_split(x_test, y_test, test_size=0.5, random_state=SEED)  # noqa
    print(f"Training set  : X-{x_train.shape}, y-{y_train.shape}")
    print(f"Validation set: X-{x_val.shape}, y-{y_val.shape}")
    print(f"Test set      : X-{x_test.shape}, y-{y_test.shape}")
    return x_train, x_val, x_test, y_train, y_val, y_test

x_train, x_val, x_test, y_train, y_val, y_test = get_data()

Training set  : X-(56000, 1, 28, 28), y-(56000,)
Validation set: X-(7000, 1, 28, 28), y-(7000,)
Test set      : X-(7000, 1, 28, 28), y-(7000,)


In [3]:
from torchpy import net

learning_rate = 0.01
epochs = 15
batch_size = 32

rng = np.random.default_rng(SEED)
shuffle_index = rng.permutation(x_train.shape[0])
x_train, y_train = x_train[shuffle_index], y_train[shuffle_index]

import torchpy.architectures as arch
model = arch.CNNMnist()

# Loss function and optimizer
loss_function = net.CrossEntropyLoss()
optimizer = net.SGD(learning_rate=learning_rate)

training_losses = []
validation_losses = []

start = time.time()
for epoch in range(epochs):
    for batch, i in enumerate(range(0, x_train.shape[0], batch_size)):
        x_batch = x_train[i:i+batch_size]
        y_batch = y_train[i:i+batch_size]
        with net.training_mode():
            # Forward pass
            logits = model(x_batch)
            loss = loss_function(logits, y_batch)
            # Backward pass
            loss_grad = loss_function.backward()
            model.backward(loss_grad)
            model.step(optimizer)
        if batch % 800 == 0:
            training_losses.append(loss)
            # calculate validation loss
            val_loss = loss_function(logits=model(x_val), targets=y_val)
            validation_losses.append(val_loss)
            print(
                f"Epoch {epoch}, Batch {batch:04}, training loss {round(loss, 3)}, ",
                f"validation loss {round(val_loss, 3)} - {time.time() - start:.2f}s",
            )

Epoch 0, Batch 0000, training loss 2.322, validation loss 2.319
Epoch 0, Batch 0800, training loss 0.099, validation loss 0.239


In [ ]:
end = time.time()
print(f"Training took {round(end - start, 2)} seconds")

In [ ]:
# plot losses over time
plt.plot(training_losses, label="Training loss")
plt.plot(validation_losses, label="Validation loss")
plt.ylim(0, 3)
plt.legend()

In [ ]:
logits = model(x_test)
loss = loss_function(logits, y_test)
print(f"Test loss: {loss}")
# get index of highest probability in each row
predicted = np.argmax(logits, axis=1)
accuracy = (predicted == y_test).sum() / y_test.size
print(f"Test accuracy: {accuracy:.3f}")

In [ ]:
from sklearn.metrics import confusion_matrix
import seaborn as sns

cm = confusion_matrix(y_test, predicted)
# remove the diagonal values (correct predictions) for better visualization
np.fill_diagonal(cm, 0)
fig = plt.figure(figsize=(OUTPUT_SIZE, OUTPUT_SIZE))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', cbar=False)
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.title('Count of Misclassified Samples by Class')

---